In [1]:
import sys
sys.path.append('./src')
import numpy as np
import torch
torch.manual_seed(0)
import torch.nn as nn
from torch.autograd import Variable,grad
from src.model import FNN
from src.util import *
from src.train import *

In [2]:
def PDE(x,y,z,t,net):
    X = torch.cat([x,y,z,t],axis=-1)
    T = net(X)
    
    T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]

    T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x,x,create_graph=True,grad_outputs=torch.ones_like(T_x))[0]
    
    T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y,y,create_graph=True,grad_outputs=torch.ones_like(T_y))[0]
    
    T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z,z,create_graph=True,grad_outputs=torch.ones_like(T_z))[0]
    
    f = rho*Cp*T_t - k*(T_xx+T_yy+T_zz)

    return f

In [3]:
def generate_points(p=[],f=[]):

    t = np.linspace(x_min[3]+0.01,x_max[3],61)

    # boundary points
    bound_x_neg,_ = sampling_uniform(1.,x_min,x_max,'-x',t)
    bound_x_pos,_ = sampling_uniform(1.,x_min,x_max,'+x',t)

    bound_y_neg,_ = sampling_uniform(1.,x_min,x_max,'-y',t)
    bound_y_pos,_ = sampling_uniform(1.,x_min,x_max,'+y',t)

    bound_z_neg,_ = sampling_uniform(1.,x_min,x_max,'-z',t)
    bound_z_pos,_ = sampling_uniform(1.,x_min,x_max,'+z',t)

    bound_z_pos_more = [] # more points for surface flux
    
    for ti in t:
        if ti<=t_end:
            zi,_ = sampling_uniform(.25,
                        [max(x0+ti*v-2*r,x_min[0]),max(x_min[1],y0-2*r),x_min[2]],
                        [min(x0+ti*v+2*r,x_max[0]),min(x_max[1],y0+2*r),x_max[2]],
                        '+z',[ti])
            bound_z_pos_more.append(zi)

    bound_z_pos_more = np.vstack(bound_z_pos_more)
    bound_z_pos = np.vstack((bound_z_pos,bound_z_pos_more))

    ### domain points
    domain_pts1,_ = sampling_uniform(2.,
                                     [x_min[0],x_min[1],x_min[2]],
                                     [x_max[0],x_max[1],x_max[2]-3.],'domain',t)

    domain_pts2,_ = sampling_uniform(1.,
                                     [x_min[0],x_min[1],x_max[2]-3.+.5],
                                     [x_max[0],x_max[1],x_max[2]-1.],'domain',t)

    domain_pts3 = []
    for ti in t:
        di,_ = sampling_uniform(.5,
                                [x_min[0],x_min[1],x_max[2]-1.+.25,],
                                [x_max[0],x_max[1],x_max[2]],'domain',[ti])
        domain_pts3.append(di)
    domain_pts3 = np.vstack(domain_pts3)
    domain_pts = np.vstack((domain_pts1,domain_pts2,domain_pts3))

    # initial points
    init_pts1,_ = sampling_uniform(2.,[x_min[0],x_min[1],x_min[2]],
                                   [x_max[0],x_max[1],x_max[2]],'domain',[0],e=0)
    # more points near the toolpath origin
    init_pts2,_ = sampling_uniform(.5,[x0-2,y0-2,x_max[2]-2],
                                   [x0+2,y0+2,x_max[2]],'domain',[0])
    
    init_pts = np.vstack((init_pts1,init_pts2))
    

    p.extend([torch.tensor(bound_x_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_x_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(init_pts,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(domain_pts,requires_grad=True,dtype=torch.float).to(device)])
    f.extend([['BC','-x'],['BC','+x'],['BC','-y'],['BC','+y'],['BC','-z'],['BC','+z'],['IC',T_ref],['domain']])
    
    return p,f

In [4]:
def BC(x,y,z,t,net,loc):
    X = torch.cat([x,y,z,t],axis=-1)
    T = net(X)
    if loc == '-x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_x - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '+x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_x - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '-y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_y - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '+y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_y - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '-z':
        T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return T_t
    if loc == '+z':
        T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        q = 2*P*eta/3.14159265/r**2*torch.exp(-2*(torch.square(x-x0-v*t)+torch.square(y-y0))/r**2)*(t<=t_end)*(t>0)
        return -k*T_z - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4) + q

In [5]:
def output_transform(X):
    X = T_range*nn.Softplus()(X)+ T_ref
    return X


def input_transform(X):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# domain
x_max = np.array([40., 10., 6., 3.])
x_min = np.array([0. ,  0., 0., 0.])
X_max = torch.tensor(x_max,dtype=torch.float).to(device)
X_min = torch.tensor(x_min,dtype=torch.float).to(device)
    
# laser params
x0 = 5.
y0 = 5.
r = 1.5
v = 10. 
t_end = 3.
P = 500.
eta = .4

# T_ambient, and max T range
T_ref = 298.
T_range = 3000.

# material params
Cp = .5
k = .01
h = 2e-5
Rboltz = 5.6704e-14
emiss = .3
rho = 8e-3
    
# valid data
data = np.load('./data/1_forward/data.npy')
test_in = torch.tensor(data[:,0:4],requires_grad=False,dtype=torch.float).to(device)
test_out = torch.tensor(data[:,4:5],requires_grad=False,dtype=torch.float).to(device)
    
    
iterations = 50000
lr = 2e-5

net = FNN([4,64,64,64,1],nn.Tanh(),in_tf=input_transform,out_tf=output_transform)
net.to(device)

point_sets,flags = generate_points([],[])

l_history,err_history = train(net,PDE,BC,point_sets,flags,iterations,lr=lr,info_num=100,
                              test_in = test_in,test_out = test_out,w=[1.,1e-4,1.,1e-4],inv_params = [])

torch.save(net.state_dict(),'./results/1_forward/no_auxilary_data.pt')
np.save('./results/1_forward/no_auxilary_data_loss.npy',l_history)
np.save('./results/1_forward/no_auxilary_data_error.npy',err_history)

It: 0, Loss: 7.908e+03, BC: 2.314e+04, IC: 5.829e+06, PDE: 1.652e+00, Test: 4.669e+06, Time: 0.32
It: 100, Loss: 5.986e+02, BC: 1.389e+03, IC: 4.062e+06, PDE: 2.149e-01, Test: 4.323e+06, Time: 23.99
It: 200, Loss: 3.412e+02, BC: 6.527e+02, IC: 3.707e+06, PDE: 8.680e-02, Test: 4.173e+06, Time: 24.01
It: 300, Loss: 2.861e+02, BC: 4.986e+02, IC: 3.596e+06, PDE: 6.264e-02, Test: 4.017e+06, Time: 24.08
It: 400, Loss: 2.440e+02, BC: 3.817e+02, IC: 3.502e+06, PDE: 4.660e-02, Test: 3.879e+06, Time: 24.33
It: 500, Loss: 2.120e+02, BC: 2.946e+02, IC: 3.415e+06, PDE: 3.531e-02, Test: 3.758e+06, Time: 24.38
It: 600, Loss: 1.879e+02, BC: 2.303e+02, IC: 3.333e+06, PDE: 2.744e-02, Test: 3.650e+06, Time: 24.40
It: 700, Loss: 1.695e+02, BC: 1.829e+02, IC: 3.256e+06, PDE: 2.197e-02, Test: 3.553e+06, Time: 24.46
It: 800, Loss: 1.553e+02, BC: 1.478e+02, IC: 3.182e+06, PDE: 1.819e-02, Test: 3.466e+06, Time: 24.45
It: 900, Loss: 1.443e+02, BC: 1.218e+02, IC: 3.109e+06, PDE: 1.558e-02, Test: 3.385e+06, Time: